In [4]:
pip install pillow


Note: you may need to restart the kernel to use updated packages.


## Trying to modify the lines using this code

In [4]:
from PIL import Image, ImageDraw

# Load the image
image_path = "/Users/rohanpadaya/Downloads/Screenshot 2024-08-02 at 2.36.22 pm.png"
img = Image.open(image_path)

# Create a drawing object
draw = ImageDraw.Draw(img)

# Example of modifying the line thickness or color
# Coordinates here are hypothetical as an example; for real use, you'd need exact line coordinates
# Modify the existing lines (for demonstration, using a set of random lines)
# Coordinates must be adapted based on the original image's content

# Draw thicker, semi-transparent lines (if coordinates are known)
draw.line((100, 100, 300, 300), fill=(255, 0, 0, 128), width=5)
draw.line((100, 300, 300, 100), fill=(0, 255, 0, 128), width=5)
draw.line((150, 150, 350, 350), fill=(0, 0, 255, 128), width=5)

# Save the modified image
modified_image_path = "/Users/rohanpadaya/Downloads/modified_image.png"
img.save(modified_image_path)

# Display the modified image (optional)
img.show()


In [6]:
pip install opencv-python-headless


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.8/54.8 MB 1.5 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


## Enhancing Visibility by Lightening Blue Edges

In [13]:
import cv2
import numpy as np
from PIL import Image

# Load the image using OpenCV
image_path = "/Users/rohanpadaya/Downloads/Screenshot 2024-08-02 at 2.36.22 pm.png"
image = cv2.imread(image_path)

# Convert the image to HSV color space
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

# Define the range of blue color in HSV
lower_blue = np.array([100, 150, 0])
upper_blue = np.array([140, 255, 255])

# Create a mask for the blue color
mask = cv2.inRange(hsv, lower_blue, upper_blue)

# Inpaint the image using the mask
inpainted_image = cv2.inpaint(image, mask, inpaintRadius=3, flags=cv2.INPAINT_TELEA)

# Convert the inpainted image back to RGB for displaying with PIL
inpainted_image_rgb = cv2.cvtColor(inpainted_image, cv2.COLOR_BGR2RGB)

# Save the resulting image using PIL
result_image_path = "/Users/rohanpadaya/Downloads/modified_image.png"
result_image = Image.fromarray(inpainted_image_rgb)
result_image.save(result_image_path)

# Display the modified image (optional)
result_image.show()


## Keeping just the nodes and the edges 

In [12]:
import cv2
import numpy as np
from PIL import Image

# Load the image using OpenCV
image_path = "/Users/rohanpadaya/Downloads/Screenshot 2024-08-02 at 2.36.22 pm.png"
image = cv2.imread(image_path)

# Convert the image to HSV color space
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

# Define the range of blue color in HSV
lower_blue = np.array([100, 150, 0])
upper_blue = np.array([140, 255, 255])

# Create a mask for the blue color
mask = cv2.inRange(hsv, lower_blue, upper_blue)

# Create an inverse mask to keep only non-blue areas
inverse_mask = cv2.bitwise_not(mask)

# Convert the entire image to white
white_background = np.ones_like(image) * 255

# Use the mask to retain the blue lines and set the rest to white
result = cv2.bitwise_and(image, image, mask=mask)
result = cv2.add(result, cv2.bitwise_and(white_background, white_background, mask=inverse_mask))

# Convert the result back to RGB for saving with PIL
result_rgb = cv2.cvtColor(result, cv2.COLOR_BGR2RGB)

# Save the resulting image using PIL
result_image_path = "/Users/rohanpadaya/Downloads/modified_image.png"
result_image = Image.fromarray(result_rgb)
result_image.save(result_image_path)

# Display the modified image (optional)
result_image.show()


# Using Graph layout algorithm to improve visibility

In [15]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from PIL import Image

# Load the image
image_path = "/Users/rohanpadaya/Downloads/Screenshot 2024-08-02 at 2.36.22 pm.png"
image = cv2.imread(image_path)

# Convert the image to HSV color space
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

# Define the range of blue color in HSV
lower_blue = np.array([100, 150, 0])
upper_blue = np.array([140, 255, 255])

# Create a mask for the blue color
mask = cv2.inRange(hsv, lower_blue, upper_blue)

# Find contours in the mask (these correspond to nodes and edges)
contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Filter the contours to find nodes (approximate as circles)
nodes = []
edges = []

for contour in contours:
    approx = cv2.approxPolyDP(contour, 0.02 * cv2.arcLength(contour, True), True)
    if len(approx) > 8:  # This is likely a node (city marker)
        M = cv2.moments(contour)
        if M['m00'] != 0:
            cx = int(M['m10'] / M['m00'])
            cy = int(M['m01'] / M['m00'])
            nodes.append((cx, cy))
    else:
        edges.append(contour)

# Create a graph from the nodes and edges
G = nx.Graph()

# Add nodes to the graph
for i, (cx, cy) in enumerate(nodes):
    G.add_node(i, pos=(cx, cy))

# Add edges to the graph
for edge in edges:
    # Find the closest node to each end of the edge
    M = cv2.moments(edge)
    if M['m00'] != 0:
        ex = int(M['m10'] / M['m00'])
        ey = int(M['m01'] / M['m00'])
        closest_nodes = sorted(nodes, key=lambda p: np.linalg.norm(np.array(p) - np.array((ex, ey))))[:2]
        if len(closest_nodes) == 2:
            idx1 = nodes.index(closest_nodes[0])
            idx2 = nodes.index(closest_nodes[1])
            G.add_edge(idx1, idx2)

# Apply a layout algorithm (e.g., spring layout) to rearrange the nodes
pos = nx.spring_layout(G)

# Draw the graph with the new layout
plt.figure(figsize=(10, 8))
nx.draw(G, pos, with_labels=True, node_color='skyblue', node_size=3000, font_size=15, font_weight='bold', edge_color='blue')

# Save the improved map
improved_map_path = "/Users/rohanpadaya/Downloads/modified_image.png"
plt.savefig(improved_map_path)
plt.close()

# Load the saved graph image and original image
graph_img = Image.open(improved_map_path)
original_img = Image.open(image_path)

# Resize the graph image to match the original image's size
graph_img = graph_img.resize(original_img.size)

# Blend the images
blended = Image.blend(original_img, graph_img, alpha=0.5)  # Adjust alpha for blending
blended.save("/Users/rohanpadaya/Downloads/modified_image.png")
blended.show()


ValueError: images do not match

As part of the assessment I tried multiple techniques to improve the visibility, however I was enable to find the most effective approach. 
* The Initial idea was to try and dim the edges to improve visibility. However, that still did not change the readibility much as the lines were overlapping. 
* The next attempt was to identify the nodes and edges from the image and redraw the map by avoiding overlaps between the edges and the distance values.Unfortunately, I failed to classify the nodes and edges from the image. I tried to classify using color and contour but was unsuccessful to do so.

The solution to this I believe lies in classifying the 